In [249]:
import numpy as np
from PIL import Image
import glob
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split

In [250]:
# Define constants
IMG_SHAPE = (64, 64)
# Download Olivetti faces dataset
olivetti = fetch_olivetti_faces()
X = olivetti.images
y = olivetti.target
# Print info on shapes and reshape where necessary
X = X.reshape((400, 4096))
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

In [251]:
def crop_image(img):
    width, height = img.size

    new_width = IMG_SHAPE[0]
    new_height = IMG_SHAPE[1]
    left = (width - new_width)/2 - .5
    top = (height - new_height)/2 - .5
    right = (width + new_width)/2 - .5
    bottom = (height + new_height)/2 - .5

    # Crop the center of the image
    im = img.crop((left, top, right, bottom))

    return im

def img_to_vector(path_to_img):
    # Load the image
    img = Image.open(path_to_img).convert('L')
    img = crop_image(img)

    # Convert the image into n²*1 array
    arr = np.array(img)
    flat_array = arr.ravel()

    return flat_array


def vector_to_img(vector, shape = IMG_SHAPE):
    array = vector.reshape(shape)
    img = Image.fromarray(array)
    return img

def sum_of_vectors(arr: []):
    sum_vector = np.zeros(len(arr[0]))
    for i in range(0, len(arr[0])):
        for v in arr:
            sum_vector[i] += v[i]

    return sum_vector


def scalar_multiply_vector(scalar, v):
    arr = np.array([])
    for x in v:
        arr = np.append(arr, x * scalar)
    return arr


def negative_vector(v):
    new_v = []
    for x in v:
        new_v.append(-x)
    return np.array(new_v)

In [252]:
all_imgs = []
training_set_photo_names = []

# Create an np.array from the vectors
training_set = Xtrain

In [253]:
# Average face using numpy
avg_face = training_set.mean(axis=0)

In [254]:
# Let's create the matrix A by subtracting the average face from each face in the training set
A = []
neg_avg_face = negative_vector(avg_face)
sub = None
for v in training_set:
    sub = np.subtract(v, avg_face)
    A.append(sub)

# Convert A to a matrix
A_m = np.asmatrix(A)

A_t = np.array(A).transpose()

In [255]:
# Form the covariance matrix
cov_matrix = np.cov(np.array(A))

In [256]:
# Calculate the eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]

In [257]:
# Choose the 10 eigenvectors with the highest eigenvalues as the eigenfaces
eigenfaces = np.array(eigvectors_sort[:20]).transpose()

In [258]:
# Calculate eiganfaces
# Create reduced eigenface space
proj_data = np.dot(training_set.transpose(), eigenfaces)
proj_data = proj_data.transpose()
# Calculate weights for eigenfaces
w = np.array([np.dot(proj_data,i) for i in np.array(A)])

In [265]:
# Get images from dataset and convert them to vectors
test_index = 20
unknown_face_vector        = Xtest[test_index]#img_to_vector('../test/Hugo_Chavez_0029.jpg')
normalized_u = np.subtract(unknown_face_vector, avg_face)
w_unknown = np.dot(proj_data, normalized_u)
difference_vector = w - w_unknown
norms = np.linalg.norm(difference_vector, axis=1)
index = np.argmin(norms)

vector_to_img(200*Xtest[test_index]).show()
vector_to_img(200*Xtrain[index]).show()
#training_set_photo_names[index]

273
